# Tutorial

In this tutorial, we are going to fine-tune the new [GPT-NeoXT-Chat-Base-20B](https://huggingface.co/togethercomputer/GPT-NeoXT-Chat-Base-20B) on the [ELI5](https://huggingface.co/datasets/eli5) dataset to improve the explanation and question-answering skills of the agent. The [ELI5](https://huggingface.co/datasets/eli5) dataset is an English-language dataset of questions and answers gathered from three subreddits where users ask factual questions requiring paragraph-length or longer answers. We are going to use Hugging Face Transformers and DeepSpeed ZeRO to fine-tune our model.



https://engineering.fb.com/2021/07/15/open-source/fsdp/
https://pytorch.org/blog/efficient-large-scale-training-with-pytorch/
https://pytorch.org/tutorials/intermediate/FSDP_tutorial.html

In [1]:
!pip install "transformers==4.26.0" "datasets[s3]==2.9.0" "sagemaker>=2.150.0" --upgrade --quiet

If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it.



In [1]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")


Couldn't call 'get_role' to get Role ARN from role name philippschmid to get Role path.


sagemaker role arn: arn:aws:iam::558105141721:role/sagemaker_execution_role
sagemaker bucket: sagemaker-us-east-1-558105141721
sagemaker session region: us-east-1


## 2. Load and prepare the dataset

As the base dataset, we will use the [ELI5](https://huggingface.co/datasets/eli5) dataset, but before fine-tuning the model, we need to preprocess the data. We will create a "chat" version of the dataset by adding `<user>` and `<bot>`tokens and add an end-of-sequence `<|endoftext|>` token to help the model learn to distinguish consecutive examples. Additionally, we create chunks of `2048` tokens ([model max length](https://huggingface.co/EleutherAI/gpt-neox-20b)) to avoid unnecessary padding and computing. 

The first step is to load our dataset from Hugging Face. The dataset contains `272634` samples for `eli5`. We will downsample the dataset to `10 000` to make it more realistic for real-world use cases.

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer 

# Load Tokenizer 
model_id = "togethercomputer/GPT-NeoXT-Chat-Base-20B"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load dataset from huggingface.co
dataset_id = "eli5"
dataset = load_dataset(dataset_id, split="train_eli5")

# downsample dataset to 10k
dataset = dataset.shuffle(42).select(range(10_000))

2023-04-27 07:11:29.998014: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 07:11:30.094125: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-27 07:11:30.096937: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-27 07:11:30.096949: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

An [ELI5](https://huggingface.co/datasets/eli5) sample can include multiple answers to a “question”. We will select the answer with the highest user score for our explanation. 

*Note: This dataset is a good example of using reinforcement learning for training transformers learning to generate answers with higher scores. Let me know if you are interested in an example of that.*

In [4]:
from random import randint

# dataset template for chat conversation
template=f'''<human>: Explain like I am five: {{question}}
<bot>: {{answer}}{{eos_token}}'''

eos_token = tokenizer.eos_token 

def template_dataset(sample):
	sample["text"] = template.format(
														question=sample["title"], 
														answer=sample["answers"]["text"][0],
														eos_token=eos_token
													)
	return sample

# apply prompt template per sample
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))

# print random sample
print(dataset[randint(0, 10_000)])

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa/cache-8abed608d38c178b.arrow


{'text': "<human>: Explain like I am five: If electricity is made from mostly fossil fuels, how is it considered a clean energy source? (e.g. Electric cars)\n<bot>: Three main reasons. First, you can get electricity from things like nuclear power, wind, hydroelectric, and solar power. It might come from fossil fuel or it might not.\n\nSecond, converting fuel into energy with low emissions is performed better with more specialized equipment. A coal power plant can scrub exhaust air with mechanisms that are impractical to carry around in every car.\n\nFinally the third and one of the most important reasons is *where* the emissions are released. If you are dumping car exhaust downtown then air quality suffers locally, but if you dump it from smokestacks on the edge of town it is much less of a problem because people aren't breathing it in as much.<|endoftext|>"}


The last step of the data preparation is to tokenize and chunk our dataset. We convert our inputs (text) to token IDs by tokenizing, which the model can understand. Additionally, we concatenate our dataset samples into chunks of `2048` to avoid unnecessary padding.

In [5]:
from itertools import chain
from functools import partial

# empty list to save remainder from batches to use in next batch
remainder = {"input_ids": [], "attention_mask": []}

def chunk(sample, chunk_length=2048):
    # define global remainder variable to save remainder from batches to use in next batch
    global remainder
    # Concatenate all texts and add remainder from previous batch
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    # get total number of tokens for batch
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # get max number of chunks for batch
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # Split by chunks of max_len.
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # add remainder to global variable for next batch
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result


# tokenize and chunk dataset
lm_dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
).map(
    partial(chunk, chunk_length=2048),
    batched=True,
)

# Print total number of samples
print(f"Total number of samples: {len(lm_dataset)}")

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa/cache-c4de07e830066b12.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa/cache-276409c2387343d5.arrow


Total number of samples: 902


After we processed the datasets we are going to use the new [FileSystem integration](https://huggingface.co/docs/datasets/filesystems) to upload our dataset to S3. We are using the `sess.default_bucket()`, adjust this if you want to store the dataset in a different S3 bucket. We will use the S3 path later in our training script.

In [6]:
# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/processed/eli-5/train'
lm_dataset.save_to_disk(training_input_path)

print("uploaded data to:")
print(f"training dataset to: {training_input_path}")

Saving the dataset (0/1 shards):   0%|          | 0/902 [00:00<?, ? examples/s]

uploaded data to:
training dataset to: s3://sagemaker-us-east-1-558105141721/processed/eli-5/train


In [2]:
training_input_path = "s3://sagemaker-us-east-1-558105141721/processed/eli-5/train"

## 3. Fine-tune the GPT model using PyTorch FSDP 

In addition to the LoRA technique, we will use [bitsanbytes LLM.int8()](https://huggingface.co/blog/hf-bitsandbytes-integration) to quantize out frozen LLM to int8. This allows us to reduce the needed memory for BLOOMZ ~4x.  

We prepared a [run_clm.py](./scripts/run_clm.py), which implements uses PEFT to train our model. If you are interested in how this works check-out [Efficient Large Language Model training with LoRA and Hugging Face](https://www.philschmid.de/fine-tune-flan-t5-peft) blog, where we explain the training script in detail. T


In order to create a sagemaker training job we need an `HuggingFace` Estimator. The Estimator handles end-to-end Amazon SageMaker training and deployment tasks. The Estimator manages the infrastructure use. 
SagMaker takes care of starting and managing all the required ec2 instances for us, provides the correct huggingface container, uploads the provided scripts and downloads the data from our S3 bucket into the container at `/opt/ml/input/data`. Then, it starts the training job by running.



In [6]:
import time
from sagemaker.huggingface import HuggingFace

# define Training Job Name 
job_name = f'huggingface-fsdp-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'


# hyperparameters, which are passed into the training job
hyperparameters={
    # 'model_id': 'togethercomputer/GPT-NeoXT-Chat-Base-20B',
    # 'model_id': 'OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5',
    # 'model_id': 'EleutherAI/pythia-12b',
    'dataset_path': '/opt/ml/input/data/training', # path where sagemaker will save training dataset
    'gradient_checkpointing': True,
    'bf16': True,
    # 'optimizer': "adafactor",
    'per_device_train_batch_size': 1,
    'epochs': 1,
    'fsdp': '"full_shard auto_wrap"',
    'fsdp_transformer_layer_cls_to_wrap': "GPTNeoXLayer",
}

hyperparameters={
    # 'model_id': 'togethercomputer/GPT-NeoXT-Chat-Base-20B',
    # 'model_id': 'OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5',
    # 'model_id': 'EleutherAI/pythia-12b',
    'model_id': 'EleutherAI/pythia-6.9b',
    'dataset_path': '/opt/ml/input/data/training', # path where sagemaker will save training dataset
    'gradient_checkpointing': True,
    'bf16': True,
    # 'optimizer': "adafactor",
    'per_device_train_batch_size': 1,
    'epochs': 1,
    'ds_config': 'ds_offload.json',
}

# estimator
huggingface_estimator = HuggingFace(
    entry_point='run_clm.py',
    source_dir='./scripts',
    instance_type="ml.p4d.24xlarge",
    instance_count=1,
    volume_size=200,
    role=role,
    job_name=job_name,
    transformers_version='4.26.0',
    pytorch_version='1.13.1',
    py_version="py39",
    hyperparameters = hyperparameters,
    distribution={"torch_distributed": {"enabled": True}}
)

# starting the train job
# huggingface_estimator.fit()

We can now start our training job, with the `.fit()` method passing our S3 path to the training script.

In [7]:
# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=True)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2023-04-28-08-32-57-582


2023-04-28 08:32:59 Starting - Starting the training job......
2023-04-28 08:33:40 Starting - Preparing the instances for training.........
2023-04-28 08:35:27 Downloading - Downloading input data
2023-04-28 08:35:27 Training - Downloading the training image..................
2023-04-28 08:38:28 Training - Training image download completed. Training in progress.......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-04-28 08:39:28,745 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-04-28 08:39:28,807 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-28 08:39:28,816 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-04-28 08:39:28,818 sagemaker_pytorch_container.training INFO     Invoking TorchDistributed...
2023-04-28 08:39:28,818 sagemaker_pytorch_container.training INFO     Invoking

UnexpectedStatusException: Error for Training job huggingface-pytorch-training-2023-04-28-08-32-57-582: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "RuntimeError: Expected all tensors to be on the same device, but found at least
 two devices, cuda:1 and cuda:0! (when checking argument for argument index in
 method wrapper__index_select)
 ╭───────────────────── Traceback (most recent call last) ──────────────────────╮
 │ /opt/ml/code/run_clm.py:144 in <module>                                      │
 │                                                                              │
 │   141                                                                        │
 │   142                                                                        │
 │   143 if __name__ == "__main__":                                             │
 │ ❱ 144 │   main()                                                             │
 │   145                                           

The trainign took `20632` seconds, which is about `5.7` hours. The `ml.g5.2xlarge` instance we used costs `$1.515` per hour. So the total cost for training BLOOMZ 7B was is `$8.63`. We could reduce the cost by using a spot instance, but the training time could increase, by waiting or restarts.